In [32]:
import pandas as pd
import psycopg2 as psy

In [33]:
# Reading the dataset
df = pd.read_csv(r'../dataModelling/Covide_DS/country_wise_latest.csv')
df.head(5)

df.describe()  # Basic aggregations

df.info()  # Print Schema

In [34]:
# Some countries have a ' in their name 
for i in range(len(df)):
    if str(df.iloc[i]['Country/Region']).find("'") != -1:
        newVal = str(df.iloc[i]['Country/Region']).replace("'", "''")
        df.at[i, 'Country/Region'] = newVal

In [35]:
# Change the inf cells to NULL
cols = df.columns

for i in range(len(df)):
    for col in cols:
        if str(df.iloc[i][col]) == 'inf':
            df.at[i, col] = 'NULL'

In [13]:
# making seperate tables out of the data
country = df[['Country/Region', 'WHO Region']]

currSituation = df[['Country/Region', 'Confirmed', 'Deaths', 'Recovered', 'Active']]

weekChange = df[['Country/Region', 'Confirmed last week', '1 week change', '1 week % increase']]

newData = df[['Country/Region', 'New cases', 'New deaths', 'New recovered']]

perHundredData = df[['Country/Region', 'Deaths / 100 Cases', 'Recovered / 100 Cases', 'Deaths / 100 Recovered']]

In [14]:
# Connect to dataBase 
def connect(connData):
    try:
        conn = psy.connect(f"host={connData['host']} dbname={connData['dbname']} user={connData['user']} password={connData['password']}")
        print(f'Connected to DB: {connData["dbname"]} .......')
    except psy.Error as e:
        print('ERROR: Unable to establish a Connection!!!')
        print(e)
    
    if conn:
        try:
            cur = conn.cursor()  # Creating cursor to execure commands
            conn.set_session(autocommit=True) # Setting autoCommit
        except psy.Error as e:
            print(f'ERROR: {e}')
            cur = None

        if cur:
            return conn, cur
        
    

In [68]:
connData = {
    'host': '127.0.0.1',
    'dbname': 'bitansarkar',
    'user': 'bitansarkar',
    'password': 'bits'
}

conn, cur = connect(connData)

Connected to DB: bitansarkar .......


In [73]:
# Creating a new Database 
try:
    cur.execute("CREATE DATABASE covid_data")
    print('Database created ...')
except psy.Error as e:
    print(e)

database "covid_data" already exists



In [74]:
# Closing the current connection
try:
    conn.close()
except psy.Error as e:
    print(e)

In [16]:
connData = {
    'host': '127.0.0.1',
    'dbname': 'covid_data',
    'user': 'bitansarkar',
    'password': 'bits'
}

conn, cur = connect(connData)

Connected to DB: covid_data .......


In [28]:
tableSchema = {
     'country': '( country varchar(200), WHO_region varchar(200))'
    ,'curr_situation': '( country varchar(200), Confirmed int, Deaths int, Recovered int, Active int)'
    ,'week_change': '( country varchar(200), Confirmed_lw int, week_chanhe int, percent_increase numeric(15,8))'
    ,'new_data': '( country varchar(200), new_cases int, new_deaths int, new_ecovered int)'
    ,'ph_data': '( country varchar(200), Deaths_PerHundred numeric(15,8), Recovered_PH numeric(15,8), Deaths_PH_Recovered numeric(15,8))'
}

In [29]:
# Create table from table list
for tableName, schema in tableSchema.items():
    sql = f'CREATE TABLE IF NOT EXISTS {tableName} {schema}'
    try:
        cur.execute(sql)
        print('Table created Successfully ..')
    except psy.Error as e:
        print(e)

Table created Successfully ..
Table created Successfully ..
Table created Successfully ..
Table created Successfully ..
Table created Successfully ..


In [30]:
# Populating Data into the tables from the DataFrame
sqlCountry = 'INSERT INTO country VALUES'
sqlCS = 'INSERT INTO curr_situation VALUES'
sqlWC = 'INSERT INTO week_change VALUES'
sqlND = 'INSERT INTO new_data VALUES'
sqlPD = 'INSERT INTO ph_data VALUES'

for i in range(len(country)):
    sqlCountry += f" ('{country.iloc[i]['Country/Region']}', '{country.iloc[i]['WHO Region']}'),"
    sqlCS += f" ('{currSituation.iloc[i]['Country/Region']}', {currSituation.iloc[i]['Confirmed']}, {currSituation.iloc[i]['Deaths']}, {currSituation.iloc[i]['Recovered']}, {currSituation.iloc[i]['Active']}),"
    sqlWC += f" ('{weekChange.iloc[i]['Country/Region']}', {weekChange.iloc[i]['Confirmed last week']}, {weekChange.iloc[i]['1 week change']}, {weekChange.iloc[i]['1 week % increase']}),"
    sqlND += f" ('{newData.iloc[i]['Country/Region']}', {newData.iloc[i]['New cases']}, {newData.iloc[i]['New deaths']}, {newData.iloc[i]['New recovered']}),"
    sqlPD += f" ('{perHundredData.iloc[i]['Country/Region']}', {perHundredData.iloc[i]['Deaths / 100 Cases']}, {perHundredData.iloc[i]['Recovered / 100 Cases']}, {perHundredData.iloc[i]['Deaths / 100 Recovered']}),"

sqlCountry = sqlCountry[:-1]
sqlCS = sqlCS[:-1]
sqlWC = sqlWC[:-1]
sqlND = sqlND[:-1] 
sqlPD = sqlPD[:-1]

In [20]:
def executeSQL(cur, sql):
    try:
        cur.execute(sql)
        print('Successfully Executed')
    except psy.Error as e:
        print(e)

In [ ]:
# Inserting all the data
executeSQL(cur, sqlCountry)
executeSQL(cur, sqlCS)
executeSQL(cur, sqlWC)
executeSQL(cur, sqlND)
executeSQL(cur, sqlPD)

In [27]:
# Drop 
def dropAllTables(cur):
    try:
        executeSQL(cur, 'DROP TABLE country')
        executeSQL(cur, 'DROP TABLE curr_situation')
        executeSQL(cur, 'DROP TABLE week_change')
        executeSQL(cur, 'DROP TABLE new_data')
        executeSQL(cur, 'DROP TABLE ph_data')
    except psy.Error as e:
        print(e)

dropAllTables(cur)

Successfully Executed
Successfully Executed
Successfully Executed
Successfully Executed
Successfully Executed


In [26]:
# Truncate 
def truncateAllTables(cur):
    try:
        executeSQL(cur, 'TRUNCATE TABLE country')
        executeSQL(cur, 'TRUNCATE TABLE curr_situation')
        executeSQL(cur, 'TRUNCATE TABLE week_change')
        executeSQL(cur, 'TRUNCATE TABLE new_data')
        executeSQL(cur, 'TRUNCATE TABLE ph_data')
    except psy.Error as e:
        print(e)

truncateAllTables(cur)

Successfully Executed
Successfully Executed
Successfully Executed
Successfully Executed
Successfully Executed
